[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# Fine-tune Qwen2.5-VL on JSON Data Extraction Dataset

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/how-to-finetune-qwen2-5-vl-for-json-data-extraction.ipynb)
[![dataset](https://app.roboflow.com/images/download-dataset-badge.svg)](https://universe.roboflow.com/roboflow-jvuqo/pallet-load-manifest)
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/QwenLM/Qwen2.5-VL)

Qwen2.5-VL is the latest vision-language model in the Qwen series, delivering state-of-the-art capabilities for understanding and analyzing images, text, and documents. Available in three sizes (3B, 7B, and 72B), it excels in tasks such as precise object localization with bounding boxes, enhanced OCR for multi-language and multi-orientation text recognition, and structured data extraction from formats like invoices, forms, and tables. With advanced image recognition spanning plants, animals, landmarks, and products, Qwen2.5-VL sets a new benchmark for multimodal understanding, catering to diverse domains including finance, commerce, and digital intelligence.

Designed for efficiency and accuracy, Qwen2.5-VL introduces innovations such as enhanced visual encoders with dynamic resolution ViT and Window Attention for computational efficiency. The 72B-Instruct model achieves competitive performance in benchmarks like document understanding and reasoning, outperforming models of similar sizes without task-specific fine-tuning. The smaller 3B and 7B models provide edge AI solutions while improving upon previous iterations, making Qwen2.5-VL a versatile choice for a wide range of vision-language applications.

![Qwen2.5-VL](https://storage.googleapis.com/com-roboflow-marketing/notebooks/examples/qwen2.5vl_arc.jpeg)

## Environment setup

### Configure your API keys

To fine-tune Qwen2.5-VL, you need to provide your HuggingFace Token and Roboflow API key. Follow these steps:

- Open your [`HuggingFace Settings`](https://huggingface.co/settings) page. Click `Access Tokens` then `New Token` to generate new token.
- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑).
    - Store HuggingFace Access Token under the name `HF_TOKEN`.
    - Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

### Configure your API keys

To fine-tune Qwen2.5-VL, you need to provide your HuggingFace Token and Roboflow API key. Follow these steps:

- Open your [`HuggingFace Settings`](https://huggingface.co/settings) page. Click `Access Tokens` then `New Token` to generate new token.
- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑).
    - Store HuggingFace Access Token under the name `HF_TOKEN`.
    - Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

In [8]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY")

### Check GPU availability

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `T4 GPU`, and then click `Save`.

In [9]:
!nvidia-smi

Mon Sep 29 21:24:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             58W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### Install dependencies

In [10]:
!pip install -q "transformers>=4.49.0" accelerate peft bitsandbytes "qwen-vl-utils[decord]==0.0.8"

## Download and prepare dataset

In [11]:
!pip install -q "roboflow>=1.1.54"

In [ ]:
from roboflow import download_dataset

dataset = download_dataset("https://app.roboflow.com/roboflow-jvuqo/pallet-load-manifest-json/2", "jsonl")

In [ ]:
!head -n 5 {dataset.location}/train/annotations.jsonl

In [ ]:
!sed -i 's/<JSON>/extract data in JSON format/g' {dataset.location}/train/annotations.jsonl
!sed -i 's/<JSON>/extract data in JSON format/g' {dataset.location}/valid/annotations.jsonl
!sed -i 's/<JSON>/extract data in JSON format/g' {dataset.location}/test/annotations.jsonl

In [ ]:
!head -n 5 {dataset.location}/train/annotations.jsonl

## Data formatting

In [ ]:
SYSTEM_MESSAGE = """You are a Vision Language Model specialized in extracting structured data from visual representations of palette manifests.
Your task is to analyze the provided image of a palette manifest and extract the relevant information into a well-structured JSON format.
The palette manifest includes details such as item names, quantities, dimensions, weights, and other attributes.
Focus on identifying key data fields and ensuring the output adheres to the requested JSON structure.
Provide only the JSON output based on the extracted information. Avoid additional explanations or comments."""

In [ ]:
def format_data(image_directory_path, entry):
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": SYSTEM_MESSAGE}],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_directory_path + "/" + entry["image"],
                },
                {
                    "type": "text",
                    "text": entry["prefix"],
                },
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": entry["suffix"]}],
        },
    ]


In [ ]:
import os
import json
import random
from PIL import Image
from torch.utils.data import Dataset


class JSONLDataset(Dataset):
    def __init__(self, jsonl_file_path: str, image_directory_path: str):
        self.jsonl_file_path = jsonl_file_path
        self.image_directory_path = image_directory_path
        self.entries = self._load_entries()

    def _load_entries(self):
        entries = []
        with open(self.jsonl_file_path, 'r') as file:
            for line in file:
                data = json.loads(line)
                entries.append(data)
        return entries

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx: int):
        if idx < 0 or idx >= len(self.entries):
            raise IndexError("Index out of range")

        entry = self.entries[idx]
        image_path = os.path.join(self.image_directory_path, entry['image'])
        image = Image.open(image_path)
        return image, entry, format_data(self.image_directory_path, entry)

In [ ]:
train_dataset = JSONLDataset(
    jsonl_file_path=f"{dataset.location}/train/annotations.jsonl",
    image_directory_path=f"{dataset.location}/train",
)
valid_dataset = JSONLDataset(
    jsonl_file_path=f"{dataset.location}/valid/annotations.jsonl",
    image_directory_path=f"{dataset.location}/valid",
)
test_dataset = JSONLDataset(
    jsonl_file_path=f"{dataset.location}/test/annotations.jsonl",
    image_directory_path=f"{dataset.location}/test",
)

In [ ]:
train_dataset[0]

## Model loading and configuration

In this section, we load the **Qwen2.5-VL** model with optional **LoRA** (Low-Rank Adaptation) or **QLoRA** (Quantized LoRA). LoRA injects small trainable weights (the “low-rank matrices”) into the model’s layers, saving significant memory and compute compared to fine-tuning the entire model. QLoRA further applies **4-bit quantization** to reduce memory usage while still preserving much of the model’s performance.

In [ ]:
import torch

from peft import get_peft_model, LoraConfig
from transformers import BitsAndBytesConfig
from transformers import Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLProcessor


MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_QLORA = True


lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)


if USE_QLORA:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_type=torch.bfloat16
    )


model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config if USE_QLORA else None,
    torch_dtype=torch.bfloat16)


model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

The model supports a wide range of resolution inputs. By default, it uses the native resolution for input, but higher resolutions can enhance performance at the cost of more computation. Users can set the minimum and maximum number of pixels to achieve an optimal configuration for their needs, such as a token count range of 256-1280, to balance speed and memory usage.

In [ ]:
MIN_PIXELS = 256 * 28 * 28
MAX_PIXELS = 1280 * 28 * 28

processor = Qwen2_5_VLProcessor.from_pretrained(MODEL_ID, min_pixels=MIN_PIXELS, max_pixels=MAX_PIXELS)

## Data collation and tokenization

### Train Collate Function
We use a masking technique that replaces padding tokens, special image tokens, and tokens from system and user turns with -100 so that the loss function ignores them and only evaluates the assistant's response.

In [ ]:
from qwen_vl_utils import process_vision_info


def train_collate_fn(batch):
    _, _, examples = zip(*batch)

    texts = [
        processor.apply_chat_template(example, tokenize=False)
        for example
        in examples
    ]
    image_inputs = [
        process_vision_info(example)[0]
        for example
        in examples
    ]

    model_inputs = processor(
        text=texts,
        images=image_inputs,
        return_tensors="pt",
        padding=True
    )

    labels = model_inputs["input_ids"].clone()

    # mask system message and image token IDs in the labels
    for i, example in enumerate(examples):
        sysuser_conv = example[:-1]
        sysuser_text = processor.apply_chat_template(sysuser_conv, tokenize=False)
        sysuser_img, _ = process_vision_info(sysuser_conv)

        sysuser_inputs = processor(
            text=[sysuser_text],
            images=[sysuser_img],
            return_tensors="pt",
            padding=True,
        )

        sysuser_len = sysuser_inputs["input_ids"].shape[1]
        labels[i, :sysuser_len] = -100

    input_ids = model_inputs["input_ids"]
    attention_mask = model_inputs["attention_mask"]
    pixel_values = model_inputs["pixel_values"]
    image_grid_thw = model_inputs["image_grid_thw"]

    return input_ids, attention_mask, pixel_values, image_grid_thw, labels

### Evaluation Collate Function

In the evaluation function, we separate out the suffixes (i.e., ground-truth target text) for later comparison. The line `examples = [e[:2] for e in examples]` effectively drops the assistant’s final turn from the template so that we only feed the model system + user and then generate the assistant output ourselves.

In [ ]:
def evaluation_collate_fn(batch):
    _, data, examples = zip(*batch)
    suffixes = [d["suffix"] for d in data]

    # drop the assistant portion so the model must generate it
    examples = [e[:2] for e in examples]

    texts = [
        processor.apply_chat_template(example, tokenize=False)
        for example
        in examples
    ]
    image_inputs = [
        process_vision_info(example)[0]
        for example
        in examples
    ]

    model_inputs = processor(
        text=texts,
        images=image_inputs,
        return_tensors="pt",
        padding=True
    )

    input_ids = model_inputs["input_ids"]
    attention_mask = model_inputs["attention_mask"]
    pixel_values = model_inputs["pixel_values"]
    image_grid_thw = model_inputs["image_grid_thw"]

    return input_ids, attention_mask, pixel_values, image_grid_thw, suffixes

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 1
NUM_WORKERS = 0

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=train_collate_fn, num_workers=NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, collate_fn=evaluation_collate_fn, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=evaluation_collate_fn, num_workers=NUM_WORKERS)

In [ ]:
input_ids, attention_mask, pixel_values, image_grid_thw, labels = next(iter(train_loader))

In [ ]:
input_ids

In [ ]:
processor.batch_decode(input_ids)

In [ ]:
torch.set_printoptions(profile="full")

In [ ]:
attention_mask

In [ ]:
labels

In [ ]:
for input_id, label in zip(input_ids[0], labels[0]):
    print(processor.decode(input_id), label.item() if label.item() == -100 else processor.decode(label))

In [ ]:
input_ids, attention_mask, pixel_values, image_grid_thw, suffixes = next(iter(valid_loader))

In [ ]:
input_ids

In [ ]:
processor.batch_decode(input_ids)

## Training with PyTorch Lightning

In [ ]:
!pip install -q lightning nltk

The `edit_distance` (from `nltk`) measures how many character-level edits (insertion, deletion, substitution) are needed to transform the predicted text into the ground truth. We normalize it by the length of the longer string to get a score between 0 (identical) and 1 (completely different).

In [ ]:
import lightning as L
from nltk import edit_distance
from torch.optim import AdamW


class Qwen2_5_Trainer(L.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, pixel_values, image_grid_thw, labels = batch
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            image_grid_thw=image_grid_thw,
            labels=labels
        )
        loss = outputs.loss
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        input_ids, attention_mask, pixel_values, image_grid_thw, suffixes = batch
        generated_ids = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            image_grid_thw=image_grid_thw,
            max_new_tokens=1024
        )
        generated_ids_trimmed = [
            out_ids[len(in_ids) :]
            for in_ids, out_ids
            in zip(input_ids, generated_ids)]

        generated_suffixes = processor.batch_decode(
            generated_ids_trimmed,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )

        scores = []
        for generated_suffix, suffix in zip(generated_suffixes, suffixes):
            score = edit_distance(generated_suffix, suffix)
            score = score / max(len(generated_suffix), len(suffix))
            scores.append(score)

            print("generated_suffix", generated_suffix)
            print("suffix", suffix)
            print("score", score)

        score = sum(scores) / len(scores)
        self.log("val_edit_distance", score, prog_bar=True, logger=True, batch_size=self.config.get("batch_size"))
        return scores

    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=self.config.get("lr"))
        return optimizer

    def train_dataloader(self):
        return DataLoader(
            train_dataset,
            batch_size=self.config.get("batch_size"),
            collate_fn=train_collate_fn,
            shuffle=True,
            num_workers=10,
        )

    def val_dataloader(self):
        return DataLoader(
            valid_dataset,
            batch_size=self.config.get("batch_size"),
            collate_fn=evaluation_collate_fn,
            num_workers=10,
        )

In [ ]:
config = {
    "max_epochs": 10,
    "batch_size": 4,
    "lr": 2e-4,
    "check_val_every_n_epoch": 2,
    "gradient_clip_val": 1.0,
    "accumulate_grad_batches": 8,
    "num_nodes": 1,
    "warmup_steps": 50,
    "result_path": "qwen2.5-3b-instruct-palette-manifest"
}

In [ ]:
model_module = Qwen2_5_Trainer(config, processor, model)

In [ ]:
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

early_stopping_callback = EarlyStopping(monitor="val_edit_distance", patience=3, verbose=False, mode="min")


class SaveCheckpoint(Callback):
    def __init__(self, result_path):
        self.result_path = result_path
        self.epoch = 0

    def on_train_epoch_end(self, trainer, pl_module):
        checkpoint_path = f"{self.result_path}/{self.epoch}"
        os.makedirs(checkpoint_path, exist_ok=True)

        pl_module.processor.save_pretrained(checkpoint_path)
        pl_module.model.save_pretrained(checkpoint_path)
        print(f"Saved checkpoint to {checkpoint_path}")

        self.epoch += 1

    def on_train_end(self, trainer, pl_module):
        checkpoint_path = f"{self.result_path}/latest"
        os.makedirs(checkpoint_path, exist_ok=True)

        pl_module.processor.save_pretrained(checkpoint_path)
        pl_module.model.save_pretrained(checkpoint_path)
        print(f"Saved checkpoint to {checkpoint_path}")

In [ ]:
trainer = L.Trainer(
    accelerator="gpu",
    devices=[0],
    max_epochs=config.get("max_epochs"),
    accumulate_grad_batches=config.get("accumulate_grad_batches"),
    check_val_every_n_epoch=config.get("check_val_every_n_epoch"),
    gradient_clip_val=config.get("gradient_clip_val"),
    limit_val_batches=1,
    num_sanity_val_steps=0,
    log_every_n_steps=10,
    callbacks=[SaveCheckpoint(result_path=config["result_path"]), early_stopping_callback],
)

trainer.fit(model_module)

### Run inference with fine-tuned Qwen2.5-VL model

In [ ]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "/content/qwen2.5-3b-instruct-palette-manifest/latest",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

processor = Qwen2_5_VLProcessor.from_pretrained(
    "/content/qwen2.5-3b-instruct-palette-manifest/latest",
    min_pixels=MIN_PIXELS,
    max_pixels=MAX_PIXELS
)

In [ ]:
def run_inference(model, processor, conversation, max_new_tokens=1024, device="cuda"):
    text = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    image_inputs, _ = process_vision_info(conversation)

    inputs = processor(
        text=[text],
        images=image_inputs,
        return_tensors="pt",
    )
    inputs = inputs.to(device)

    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_ids_trimmed = [
        out_ids[len(in_ids):]
        for in_ids, out_ids
        in zip(inputs.input_ids, generated_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    return output_text[0]

In [ ]:
image, entry, conversation = test_dataset[0]
conversation = conversation[:2]
suffix = entry["suffix"]
suffix

In [ ]:
generated_suffix = run_inference(model, processor, conversation)
generated_suffix

In [ ]:
image

In [ ]:
from difflib import SequenceMatcher
from IPython.core.display import display, HTML

def side_by_side_diff_divs(text1, text2):
    lines1 = text1.splitlines()
    lines2 = text2.splitlines()

    original_output = []
    modified_output = []

    for line1, line2 in zip(lines1, lines2):
        words1 = line1.split()
        words2 = line2.split()

        matcher = SequenceMatcher(None, words1, words2)

        original_line = []
        modified_line = []

        for tag, i1, i2, j1, j2 in matcher.get_opcodes():
            if tag == 'replace':
                original_line.append(f"<span class='diff-remove'>{' '.join(words1[i1:i2])}</span>")
                modified_line.append(f"<span class='diff-add'>{' '.join(words2[j1:j2])}</span>")
            elif tag == 'delete':
                original_line.append(f"<span class='diff-remove'>{' '.join(words1[i1:i2])}</span>")
            elif tag == 'insert':
                modified_line.append(f"<span class='diff-add'>{' '.join(words2[j1:j2])}</span>")
            elif tag == 'equal':
                original_line.append(' '.join(words1[i1:i2]))
                modified_line.append(' '.join(words2[j1:j2]))

        original_output.append(' '.join(original_line) + "<br>")
        modified_output.append(' '.join(modified_line) + "<br>")

    original_html = "<br>" + ''.join(original_output) + "<br>"
    modified_html = "<br>" + ''.join(modified_output) + "<br>"

    html = f"""
    <html>
    <head>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 0; padding: 0; }}
            .container {{ display: flex; align-items: flex-start; }}
            .column {{
                flex: 1;
                padding: 10px;
                white-space: pre-wrap;
                text-align: left;
            }}
            .diff-remove {{
                background-color: #d9534f;  /* Dark red */
                color: white;
                text-decoration: line-through;
                border-radius: 4px;
                padding: 2px 4px;
            }}
            .diff-add {{
                background-color: #5cb85c;  /* Dark green */
                color: white;
                border-radius: 4px;
                padding: 2px 4px;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="column" style="border-right: 1px solid #ccc;">
                {original_html}
            </div>
            <div class="column">
                {modified_html}
            </div>
        </div>
    </body>
    </html>
    """
    return html

In [ ]:
html_diff = side_by_side_diff_divs(suffix, generated_suffix)
display(HTML(html_diff))